In [1]:
import pandas as pd
import fitz
import re
from pathlib import Path
import unicodedata

Extract text containing only code E

In [2]:
def normalize_whitespace(text):
    return re.sub(r'\s+', ' ', text).strip()

def normalize_unicode(text):
    return unicodedata.normalize("NFKC", text)

def parse_page_range(text):
    text = str(text)
    numbers = set()
    for part in re.split(r"[;,]", text):
        part = part.strip()
        if re.match(r"^\d+\s*[-–]\s*\d+$", part):
            start, end = map(int, re.split(r"[-–]", part))
            numbers.update(range(start, end + 1))
        elif part.isdigit():
            numbers.add(int(part))
    return numbers

def extract_pdf_text_by_code_and_section(pdf_path):
    pdf_path = Path(pdf_path)
    pdf_stem = pdf_path.stem

    table_path = Path("../2_output/standardized_merged_by_company") / f"{pdf_stem}_standardized_full.csv"
    if not table_path.exists():
        raise FileNotFoundError(f"❌ Cannot find the standardized table at: {table_path}")

    df = pd.read_csv(table_path)
    df.columns = df.columns.str.strip()
    df["Code"] = df["Code"].astype(str).str.strip()
    df["Relevant Pages"] = df["Relevant Pages"].astype(str).str.strip()
    df["Section Reference"] = df["Section Reference"].astype(str).str.strip()
    df["Page Range"] = df["Page Range"].astype(str).str.strip()

    doc = fitz.open(pdf_path)
    output = []

    for i, row in df.iterrows():
        if i == 0:
            continue  # 🆕 Skip the first row unconditionally

        code = row["Code"]
        valid_code = bool(re.search(r"\b(ESRS\s*)?E[-\s]?\d+\b", code, re.IGNORECASE))
        if not valid_code:
            continue

        rel_pages_raw = row["Relevant Pages"]
        section_raw = row["Section Reference"]
        page_range_set = parse_page_range(row.get("Page Range", ""))
        EXPANSION_RANGE = 2

        total_pages = len(doc)
        cutoff_page = int(total_pages * 0.8)

        # ✅ Relevant pages logic
        used_pages = set()
        if rel_pages_raw and pd.notna(rel_pages_raw):
            page_nums = [int(p) for p in re.findall(r"\d+", rel_pages_raw)]
            for p in page_nums:
                if 20 < p <= cutoff_page:
                    for n in range(p - EXPANSION_RANGE, p + EXPANSION_RANGE + 1):
                        if 1 <= n <= cutoff_page:
                            used_pages.add(n)
        for page_num in sorted(used_pages):
            raw_text = doc[page_num - 1].get_text()
            page_text = normalize_whitespace(normalize_unicode(raw_text))
            output.append({
                "Matched Code": code,
                "Relevant Pages (Raw)": rel_pages_raw,
                "Used Pages": page_num,
                "Section Reference": None,
                "Extracted Text": page_text.strip()
            })

        # ✅ Section Reference logic — one output per section string
        if section_raw and pd.notna(section_raw):
            section_list = [s.strip() for s in section_raw.split(",") if s.strip()]
            for sec in section_list:
                matched_pages = set()
                for page_num in range(21, total_pages + 1):
                    if page_num in page_range_set:
                        continue
                    text = doc[page_num - 1].get_text()
                    if re.search(rf"\b{re.escape(sec)}\b", text):
                        print(f"[Section Match ✅] Row {i}, Code='{code}', Section='{sec}', Triggered Page={page_num}")
                        if 20 < page_num <= cutoff_page:
                            for n in range(page_num - EXPANSION_RANGE, page_num + EXPANSION_RANGE + 1):
                                if 1 <= n <= cutoff_page:
                                    matched_pages.add(n)

                if not matched_pages:
                    print(f"[Section Miss ❌] Row {i}, Code='{code}', Section='{sec}' — no matches found.")

                for page_num in sorted(matched_pages):
                    raw_text = doc[page_num - 1].get_text()
                    page_text = normalize_whitespace(normalize_unicode(raw_text))
                    output.append({
                        "Matched Code": code,
                        "Relevant Pages (Raw)": rel_pages_raw,
                        "Used Pages": page_num,
                        "Section Reference": sec,
                        "Extracted Text": page_text.strip()
                    })


    return pd.DataFrame(output)

In [8]:
pdf_path = "../0_data/pdfs/UniCredit.pdf"
result_df = extract_pdf_text_by_code_and_section(pdf_path)

In [9]:
result_df

""


In [10]:
#preview
for i, row in result_df.iterrows():
    print(f"\n--- Code: {row['Matched Code']} | Page: {row['Used Pages']} | Section: {row['Section Reference']} ---\n")
    print(row["Extracted Text"][:1000])  # preview first 1000 chars

In [11]:
#save results in folder extracted_text_indiv
text_only = result_df["Extracted Text"]

company_name = Path(pdf_path).stem.strip()
save_path = Path("../2_output/extracted_text_indiv") / f"{company_name}_extracted_text_only.txt"
save_path.parent.mkdir(parents=True, exist_ok=True)

text_only.to_csv(save_path, index=False, header=False)
print(f"✅ Saved extracted text only to: {save_path}")

KeyError: 'Extracted Text'

Alternatively, run through all pdfs in excel

In [ ]:
# === Load all report names from Excel ===
REPO_ROOT = Path().resolve()
excel_path = REPO_ROOT.parent / "0_data" / "table_extraction_accuracy2.xlsx"
sheet_name = "suggested weights"
pdf_df = pd.read_excel(excel_path, sheet_name=sheet_name)

# Match full filename with extension
pdf_df = pdf_df.dropna(subset=["pdf_path", "emission level"])
pdf_df["pdf_name"] = pdf_df["pdf_path"].apply(lambda p: Path(p).name)
pdf_names = pdf_df["pdf_name"].unique()
emission_map = dict(zip(pdf_df["pdf_name"], pdf_df["emission level"]))

pdf_folder = Path("../0_data/pdfs")
output_folder = Path("../2_output/extracted_text_indiv")
output_folder.mkdir(parents=True, exist_ok=True)

processed_count = 0
error_count = 0
high_emission_count = 0
low_emission_count = 0

# === Process each matching PDF ===
for pdf_path in pdf_folder.glob("*.pdf"):
    if pdf_path.name in pdf_names:
        try:
            print(f"🔍 Processing: {pdf_path.name}")
            result_df = extract_pdf_text_by_code_and_section(pdf_path)
            text_only = result_df["Extracted Text"]

            save_path = output_folder / f"{pdf_path.stem}_extracted_text_only.txt"
            text_only.to_csv(save_path, index=False, header=False)
            print(f"✅ Saved: {save_path}")
            processed_count += 1

            # Count by emission category
            emission_label = emission_map.get(pdf_path.name, "").strip().lower()
            if emission_label == "high":
                high_emission_count += 1
            elif emission_label == "low":
                low_emission_count += 1

        except Exception as e:
            print(f"❌ Error processing {pdf_path.name}: {e}")
            error_count += 1
    else:
        print(f"⏩ Skipped (not in Excel): {pdf_path.name}")


🔍 Processing: UniCredit.pdf
❌ Error processing UniCredit.pdf: 'Extracted Text'
🔍 Processing: AB InBev 2024 Annual Report FINAL_Interactive.pdf
❌ Error processing AB InBev 2024 Annual Report FINAL_Interactive.pdf: ❌ Cannot find the standardized table at: ../2_output/standardized_merged_by_company/AB InBev 2024 Annual Report FINAL_Interactive_standardized_full.csv
🔍 Processing: Covestro_2024_GB_EN.pdf
[Section Miss ❌] Row 3, Code='E2', Section='nan' — no matches found.
[Section Miss ❌] Row 4, Code='E3', Section='nan' — no matches found.
❌ Error processing Covestro_2024_GB_EN.pdf: 'Extracted Text'
🔍 Processing: consolidated-annual-report-endesa-2024.pdf
❌ Error processing consolidated-annual-report-endesa-2024.pdf: 'Extracted Text'
🔍 Processing: Amorim_RC24_EN.pdf
❌ Error processing Amorim_RC24_EN.pdf: 'Extracted Text'
🔍 Processing: Valeo_2024-Universal-registration-document.pdf
[Section Match ✅] Row 1, Code='E1-3', Section='4.2.3.6', Triggered Page=210
[Section Match ✅] Row 1, Code='E1-3

In [ ]:
# === Summary ===
print(f"\n✅ Done. Total processed: {processed_count} PDF(s)")
print(f"❌ Total errors: {error_count}")
print(f"📊 High-emission reports processed: {high_emission_count}")
print(f"📊 Low-emission reports processed: {low_emission_count}")

Now create separate txt file for high and low emission firms

In [4]:
REPO_ROOT = Path().resolve()
excel_path = REPO_ROOT.parent / "0_data" / "table_extraction_accuracy2.xlsx"
sheet_name = "suggested weights"
pdf_df = pd.read_excel(excel_path, sheet_name=sheet_name)

pdf_df = pdf_df[["pdf_path", "emission level"]].dropna()
pdf_df["pdf_name"] = pdf_df["pdf_path"].apply(lambda p: Path(p).stem.strip())
emission_map = dict(zip(pdf_df["pdf_name"], pdf_df["emission level"]))

# === Input and output folders ===
input_folder = Path("../2_output/extracted_text_indiv")
output_folder = Path("../2_output/extracted_text_indiv")
output_folder.mkdir(parents=True, exist_ok=True)

# === Prepare containers ===
grouped = {"High": [], "Low": []}

# === Loop through extracted txt files ===
for file in input_folder.glob("*_extracted_text_only.txt"):
    stem = file.stem.replace("_extracted_text_only", "")
    label = emission_map.get(stem)

    if label in grouped:
        content = file.read_text(encoding="utf-8").strip()
        if content:
            grouped[label].append(f"--- {stem} ---\n{content}")
    else:
        print(f"⚠️ Skipped {stem}: No emission label found")

# === Save grouped files ===
for label, texts in grouped.items():
    out_path = output_folder / f"{label}_Emission.txt"
    out_path.write_text("\n\n".join(texts), encoding="utf-8")
    print(f"✅ Saved grouped file: {out_path}")

✅ Saved grouped file: ../2_output/extracted_text_indiv/High_Emission.txt
✅ Saved grouped file: ../2_output/extracted_text_indiv/Low_Emission.txt


Devide text on low and high emission ones

In [5]:
REPO_ROOT = Path().resolve()
excel_path = REPO_ROOT.parent / "0_data" / "table_extraction_accuracy2.xlsx"
sheet_name = "suggested weights"
pdf_df = pd.read_excel(excel_path, sheet_name=sheet_name)

pdf_df = pdf_df[["pdf_path", "emission level"]].dropna()
pdf_df["pdf_name"] = pdf_df["pdf_path"].apply(lambda p: Path(p).stem.strip())
emission_map = dict(zip(pdf_df["pdf_name"], pdf_df["emission level"]))

# === Input and output folders ===
input_folder = Path("../2_output/extracted_text_indiv")
output_folder = Path("../2_output/extracted_text_indiv")
output_folder.mkdir(parents=True, exist_ok=True)

# === Prepare containers ===
grouped = {"High": [], "Low": []}

# === Loop through extracted txt files ===
for file in input_folder.glob("*_extracted_text_only.txt"):
    stem = file.stem.replace("_extracted_text_only", "")
    label = emission_map.get(stem)

    if label in grouped:
        content = file.read_text(encoding="utf-8").strip()
        if content:
            grouped[label].append(f"--- {stem} ---\n{content}")
    else:
        print(f"⚠️ Skipped {stem}: No emission label found")

# === Save grouped files ===
for label, texts in grouped.items():
    out_path = output_folder / f"{label}_Emission.txt"
    out_path.write_text("\n\n".join(texts), encoding="utf-8")
    print(f"✅ Saved grouped file: {out_path}")

✅ Saved grouped file: ../2_output/extracted_text_indiv/High_Emission.txt
✅ Saved grouped file: ../2_output/extracted_text_indiv/Low_Emission.txt
